In [2]:
# Load iSearch's Directory Admin Tool via selenium in Chrome
from selenium import webdriver
driver = webdriver.Chrome(executable_path=r"C:\Users\tbutterf\chromedriver.exe")
driver.get("https://isearch.asu.edu/asu_dept_mgr");

In [3]:
# NEW Find/Replace Titles, based on spreadsheet of title fixes
from IPython.core.debugger import set_trace
set_trace()
import time
import pandas as pd

titlelist = pd.read_excel("titles-comprehensive.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')

for i in range(6,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    nametext = []
    nametext.append(name.text)
    name.click()
    time.sleep(10)
    box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
    workingtitle = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute('value')
#     print('working: '+workingtitle)
    customtitle = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]')
    customtitlevalue= customtitle.get_attribute('value')
#     print('custom: '+customtitlevalue)
    matchworking = titlelist[titlelist['Title']==workingtitle]
#     print('matchworking: '+matchworking)
    matchcustom = titlelist[titlelist['Title']==customtitlevalue]
#     print('matchcustom: '+matchcustom)
    fixedtitle = titlelist[titlelist['Title']==customtitlevalue]['Fixed Title'].values
    savebutton = driver.find_element_by_id('edit-save')
    cancelbutton = driver.find_element_by_id('edit-cancel')
    if(box.is_selected()):
        if(len(fixedtitle) > 0) and (customtitlevalue != fixedtitle[0]):
            customtitle.clear()
            customtitle.send_keys(fixedtitle[0])
            print(str(i)+"|"+nametext[0]+"|"+customtitlevalue+"|Newly Fixed")
            del nametext[:]
            savebutton.click()
            time.sleep(10)
        elif(len(fixedtitle) > 0) and (customtitlevalue == fixedtitle[0]):
            print(str(i)+"|"+nametext[0]+"|"+customtitlevalue+"|Already fixed")
            del nametext[:]
            cancelbutton.click()
            time.sleep(10)
        else:
            print(str(i)+"|"+nametext[0]+"|"+customtitlevalue+"|No fixed title match|boxyes")
            del nametext[:]
            cancelbutton.click()
            time.sleep(10)
        
    else:
        if(len(fixedtitle) > 0) and (workingtitle == fixedtitle[0]):
            print(str(i)+"|"+nametext[0]+"|"+workingtitle+"|Already fixed")
            del nametext[:]
            cancelbutton.click()
            time.sleep(10)
        elif(len(fixedtitle) > 0) and (workingtitle != fixedtitle[0]):
            box.click()
            customtitle.clear()
            customtitle.send_keys(fixedtitle[0])
            print(str(i)+"|"+nametext[0]+"|"+customtitlevalue+"|Newly fixed")
            del nametext[:]
            savebutton.click()
            time.sleep(10)
        else:
            print(str(i)+"|"+nametext[0]+"|"+workingtitle+"|No fixed title match|boxno")
            del nametext[:]
            cancelbutton.click()
            time.sleep(10)
            
print("Done!")

--Return--
None
> <ipython-input-3-887967e91767>(3)<module>()
      1 # NEW Find/Replace Titles, based on spreadsheet of title fixes
      2 from IPython.core.debugger import set_trace
----> 3 set_trace()
      4 import time
      5 import pandas as pd

ipdb> c
6|Matthew Ingram|Director of Development |Newly Fixed
7|Laura Thede (PUBSRV)|Associate Director, Academic Personnel|Already fixed
8|Dale Larsen|Director of Community Relations and Professor of Practice|Already fixed
9|Diane Bjorem|Office Supervisor|Already fixed
10|Brian Gerber|Associate Professor & Co-Director, Center for Emergency Management & Homeland Security (CEMHS)|Already fixed
11|Nicole Prenovost|Administrative Assistant|Already fixed
12|Michael Shafer|Professor|Already fixed

Program interrupted. (Use 'cont' to resume).
None
> <ipython-input-3-887967e91767>(18)<module>()
     16     name.click()
     17     time.sleep(10)
---> 18     box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-title

BdbQuit: 

In [ ]:
# ---ISEARCH ADD-NEW EMPLOYEE TOOL---
# Imports spreadsheet of specific unit, checks iSearch for people that have not been entered and 
# enters them programatically. Spreadsheet must have 4 columns in this order: Name, ASURITE, Title, Employee Type. 
# It also references a spreadsheet to match faculty's current titles, populates the "Faculty Rank"
# field AND fixes known PeopleSoft title errors/abbreviations when adding new people to a unit.

# from IPython.core.debugger import set_trace
# set_trace()
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
table = pd.read_excel("faculty-rank-conversion.xlsx")
unitlist = pd.read_excel("staff-retry.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
isearchnames = []
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
search = driver.find_element_by_id('edit-asu-dept-mgr-userpicker')
addbutton = driver.find_element_by_id('edit-asu-dept-mgr-add-to-dept')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    isearchnames.append(name.text)
    #print("Names in iSearch:",*isearchnames, sep='\n')

startingrow = 2 #(Row number of first person in spreadsheet, or row number of where to continue after an error)
for i in range(startingrow-2, unitlist.shape[0]):
    rowSeries = unitlist.iloc[i]
    fullname = rowSeries[0]
    asurite = rowSeries[1]
    pos_title = rowSeries[2]
    empltype = rowSeries[3]
    try:
        if(isearchnames.index(rowSeries[0])):
            print(str(i+2)+". Exists: "+fullname)
            time.sleep(2)
    except ValueError: 
        search.clear()
        search.send_keys(asurite)
        addbutton.click()
        time.sleep(20)
        print(str(i+2)+". Added: "+fullname)
        try:
            box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
            #workingtitle = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute('value')
            #print('working: '+workingtitle)
            customtitle = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]')
            #customtitlevalue = customtitle.get_attribute('value')
            #print('custom: '+customtitlevalue)
            if(not box.is_selected()):
                box.click()
            customtitle.clear()
            customtitle.send_keys(pos_title)
            time.sleep(4)
        #     def set_or_fix_rank():
        #         ftype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]')).first_selected_option.text
        # #         print(ftype)
        #         fname = driver.find_element_by_xpath('//*[@id="relation-ui-edit-form"]/div/h2')
        #         title = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute("value")
        #         rank = table[table['Title']==pos_title]['Faculty Rank'].values
        #         setempltype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]'))
        #         setempltype.select_by_visible_text(empltype)
        #         if len(rank[0])>0:
        #             if title.find('amp;')>0:
        #                 title = title.replace('amp;','')
        #             select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
        #             select.select_by_visible_text(rank[0])
        #             print(str(i+2)+". Rank updated:",fname.text,"(Rank:",rank[0]+")")
        #     set_or_fix_rank()
            savebutton = driver.find_element_by_id('edit-save')
            cancelbutton = driver.find_element_by_id('edit-cancel')
            savebutton.click()
        except:
            print(str(i+2)+". ERROR on "+fullname+" ("+asurite+"). Please doublecheck.")
            time.sleep(2)
            try:
                search.clear()
                time.sleep(6)
            except:
                savebutton.click()
                time.sleep(6)
        time.sleep(12)
print("Done!")

In [ ]:
# ---FIX PREVIOUSLY IMPORTED TITLES, FACULTY RANKS, & EMPLOYEE TYPES---
# Imports spreadsheet of specific unit, checks iSearch for people that have not been entered and 
# enters them programatically. Spreadsheet must have 4 columns in this order: Name, ASURITE, Title, Employee Type. 
# It also references a spreadsheet to match faculty's current titles, populates the "Faculty Rank" field.

# from IPython.core.debugger import set_trace
# set_trace()
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
table = pd.read_excel("faculty-rank-conversion.xlsx")
unitlist = pd.read_excel("force-check-olli-stuff.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
isearchnames = []
search = driver.find_element_by_id('edit-asu-dept-mgr-userpicker')
addbutton = driver.find_element_by_id('edit-asu-dept-mgr-add-to-dept')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div//table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    nametext = []
    nametext.append(name.text)
    pos_title = unitlist[unitlist['Name']==nametext[0]]['Title'].values
    empltype = unitlist[unitlist['Name']==nametext[0]]['Employee Type'].values
    box = row.find_elements_by_tag_name("td")[0].find_element_by_tag_name("input")
    hrname = unitlist[unitlist['Name']==nametext[0]]['Name'].values
    if(len(hrname)>0):
        name.click()
        time.sleep(10)
        box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
        workingtitle = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute('value')
        #print('working: '+workingtitle)
        customtitle = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]')
        if(not box.is_selected()):
            box.click()
        customtitle.clear()
        time.sleep(1)
        customtitle.send_keys(pos_title[0])
        print(str(i)+". Title updated:",nametext[0],"(Title:",pos_title[0]+")")
        def set_or_fix_rank():
            ftype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]')).first_selected_option.text
    #         print(ftype)
            fname = driver.find_element_by_xpath('//*[@id="relation-ui-edit-form"]/div/h2')
            title = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute("value")
            rank = table[table['Title']==pos_title[0]]['Faculty Rank'].values
            setempltype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]'))
            setempltype.select_by_visible_text(empltype[0])
            if len(rank[0])>0:
                if title.find('amp;')>0:
                    title = title.replace('amp;','')
                select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
                select.select_by_visible_text(rank[0])
                print(str(i)+". Rank updated:",fname.text,"(Rank:",rank[0]+")")
        set_or_fix_rank()
        savebutton = driver.find_element_by_id('edit-save')
        cancelbutton = driver.find_element_by_id('edit-cancel')
        savebutton.click()
        del nametext[:]
        time.sleep(10)
    else:
        continue
        del nametext[:]
         
print("Done!")

In [ ]:
# Load iSearch via selenium in Chrome for the primary affiliation tool
from selenium import webdriver
driver = webdriver.Chrome(executable_path=r"C:\Users\tbutterf\chromedriver.exe")
driver.get("https://weblogin.asu.edu/cas/login?service=https%3A%2F%2Fisearch.asu.edu%2Fcas%3Fdestination%3Dasu-people");


In [ ]:
#PRIMARY AFFILIATION TOOL
#This tool checks iSearch to see if the desired unit listed in an Excel Spreadsheet is the primary affiliation
#of the people listed in that spreadsheet. If not, it changes the affiliation accordingly. If the primary affiliation
#is for a unit outside of the college, this tool leaves the primary affiliation alone (does not change it).

import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
unitlist = pd.read_excel("crd-nontenure.xlsx")
excelrow = next(unitlist.iterrows())
for excelrow in unitlist.itertuples():
    excelname = excelrow[1]
    excelasurite = excelrow[2]
    iteration = excelrow[0] 
    excelunit = excelrow[3]
    try:
        searchbox = driver.find_element_by_xpath('//*[@id="ajaxSolrSearch"]/input[contains(@id,'+
                                                 ' "edit-is-ajax-search-text")]')
        submitsearch = driver.find_element_by_xpath('//*[@id="edit-submit"]')
        searchbox.clear()
        searchbox.send_keys(excelasurite)
        submitsearch.click()
    except NoSuchElementException:
        facultystaff = driver.find_element_by_xpath('//*[@id="is-search-people"]')
        facultystaff.click()
        time.sleep(2)
        newsearchbox = driver.find_element_by_xpath('//*[@id="is-ajax-solr-secondary-search-form"]'+
                                                    '/div/div/input[contains(@id, "edit-is-ajax-search-text")]')
        submitnewsearch = driver.find_element_by_xpath('//*[@id="edit-submit"]')
        newsearchbox.clear()
        time.sleep(1)
        newsearchbox.send_keys(excelasurite)
        submitnewsearch.click()
    print(str(iteration)+': '+excelname)
    time.sleep(2)
    row = driver.find_element_by_xpath('//*[@id="is-ajax-solr-results"]/div[contains(@class, "row")][1]')
    name = row.find_element_by_class_name("displayName")
    name.click()
    editprofile = driver.find_element_by_class_name("update-print-profile")
    editprofile.click()
    afftab = driver.find_element_by_xpath('//*[@id="is-profile-node-form"]/div/div/div[2]/div[1]/div/ul/li[3]/a')
    afftab.click()

    wattstest = driver.find_element_by_xpath('//*[@id="asu-dept-profile-affiliations"]/tbody/tr/td/'+
    'div[contains(@class, "affil-dept") and contains(text(), "Watts College of Public Service & Community Solutions")]')
    wattsrow = wattstest.find_element_by_xpath('ancestor::tr')
    wattsradio = wattsrow.find_element_by_xpath('td[2]/div/label/input[@name="dept_affils_table"]')
    wattstest = driver.find_element_by_xpath('//*[@id="asu-dept-profile-affiliations"]/tbody/tr/td/div[contains(@class,'+
                         ' "affil-dept") and contains(text(), "Watts College of Public Service & Community Solutions")]')
    wattsrow = wattstest.find_element_by_xpath('ancestor::tr')
    wattsradio = wattsrow.find_element_by_xpath('td[2]/div/label/input[@name="dept_affils_table"]')
    #Check if Watts College is the primary affiliation.
    wattschecked = wattsradio.get_attribute('checked')
    if (wattschecked == 'true'):
        #If Watts College is the primary affiliation, set 'excelunit' as primary affiliation
        afflist = driver.find_element_by_xpath('//*[@id="asu-dept-profile-affiliations"]/tbody/tr/td/'+
                'div[contains(@class, "affil-dept") and contains(text(), "'+excelunit+'")]')
        affrow = afflist.find_element_by_xpath('ancestor::tr')
        affradio = affrow.find_element_by_xpath('td[2]/div/label/input[@name="dept_affils_table"]')
        affradio.click()
        time.sleep(1)
        submit = "document.querySelector('#edit-submit').click();"
        driver.execute_script(submit)
        print(str(iteration)+': Success -- '+excelname)
        time.sleep(2)
    else:
        #If Watts College is not the primary affiliation
        afflist = driver.find_element_by_xpath('//*[@id="asu-dept-profile-affiliations"]/tbody/tr/td/'+
                'div[contains(@class, "affil-dept") and contains(text(), "'+excelunit+'")]')
        affrow = afflist.find_element_by_xpath('ancestor::tr')
        affradio = affrow.find_element_by_xpath('td[2]/div/label/input[@name="dept_affils_table"]')
        affchecked = affradio.get_attribute('checked')
        if (affchecked == 'true'):
            #Check to see if 'excelunit' variable is the primary affiliation. If so, move on to next persion.         
            viewprofile = driver.find_element_by_xpath('//*[@id="is-profile-node-form"]/div/div/div[1]/div/a')
            viewprofile.click()   
            print(str(iteration)+': No change needed. '+excelunit+'is already the primary affiliation. '+excelname)
        else:
            #If 'excelunit' and Watts College are not primary affiliation, do nothing.
            viewprofile = driver.find_element_by_xpath('//*[@id="is-profile-node-form"]/div/div/div[1]/div/a')
            viewprofile.click()   
            print(str(iteration)+': NOTICE: Some other unit is primary. Do nothing. '+excelname)     
print('Done!')

In [ ]:
# Check spreadsheet of people against what is in iSearch. Fix titles and Faculty Rank.

import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
table = pd.read_excel("faculty-rank-conversion.xlsx")
unitlist = pd.read_excel("college-staff-strict.xlsx")
titlelist = pd.read_excel("titles.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
search = driver.find_element_by_id('edit-asu-dept-mgr-userpicker')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div//table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    nametext = []
    nametext.append(name.text)
    box = row.find_elements_by_tag_name("td")[0].find_element_by_tag_name("input")
    hrname = unitlist[unitlist['Name']==nametext[0]]['Name'].values
    if(len(hrname)>0):
        name.click()
        time.sleep(14)        
        
        def fix_rank():
            try:
                ftype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]')).first_selected_option.text
        #         print(ftype)
                fname = driver.find_element_by_xpath('//*[@id="relation-ui-edit-form"]/div/h2')
                customizebox = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
                if (ftype == "Faculty") or (ftype == "Faculty w/Admin Appointment") or (ftype == "Academic Professional") or (ftype == "Academic Prof w/Admin Appointment"):
                    title = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute("value")
                    if title.find('amp;')>0:
                        title = title.replace('amp;','')
                    rank = table[table['Title']==title]['Faculty Rank'].values
                    if len(rank)>0:
                        select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
                        select.select_by_visible_text(rank[0])
                        print(str(i+2)+". Rank updated:",fname.text,"(Rank:",rank[0]+")")
                    else:
                        print(fname.text+" (Missing Rank:",title+")")
            except NoSuchElementException:
                print(str(i+2)+". ERROR fixing faculty rank"+nametext)
        
        try:
            box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
            workingtitle = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute('value')
        #     print('working: '+workingtitle)
            customtitle = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]')
            customtitlevalue= customtitle.get_attribute('value')
        #     print('custom: '+customtitlevalue)
            matchworking = titlelist[titlelist['Title']==workingtitle]
        #     print('matchworking: '+matchworking)
            matchcustom = titlelist[titlelist['Title']==customtitlevalue]
            matchfixedcustom = titlelist[titlelist['Fixed Title']==customtitlevalue]
        #     print('matchcustom: '+matchcustom)
            savebutton = driver.find_element_by_id('edit-save')
            cancelbutton = driver.find_element_by_id('edit-cancel')
            if(str(workingtitle) == "" and not box.is_selected()):
                savebutton.click()
                print("A|"+str(i)+"|"+nametext[0]+'|current working title: |'+workingtitle+'||No change made')
                fix_rank()
                del nametext[:] 
                time.sleep(10)
            elif(len(matchworking)>0):
                if(not box.is_selected()):
                    box.click()
                    time.sleep(2)
                    savebutton = driver.find_element_by_id('edit-save')
                    newtitle = matchworking['Fixed Title'].values[0]
                    customtitle.clear()
                    customtitle.send_keys(newtitle)
                    savebutton.click()
                    print("B|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '+workingtitle)
                    fix_rank()
                    del nametext[:] 
                    time.sleep(10)
                elif(box.is_selected()):
                    try:
                        newtitle = matchcustom['Fixed Title'].values[0]
                        if(customtitle!=newtitle):
                            customtitle.clear()
                            customtitle.send_keys(newtitle)
                            savebutton.click()
                            print("C|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '
                                  +workingtitle)
                            fix_rank()
                            del nametext[:] 
                            time.sleep(10)
                        else:
                            savebutton.click()
                            print("D|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+'||No change made')
                            fix_rank()
                            del nametext[:] 
                            time.sleep(10)
                    except:
                        cancelbutton.click()
                        print("E|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+
                              '|old working title: '+workingtitle+'|No change made')
                        fix_rank()
                        del nametext[:] 
                        time.sleep(10)
            elif(len(matchcustom)>0):
                if(box.is_selected()):
                    try:
                        newtitle = matchcustom['Fixed Title'].values[0]
            #             print('newtitle: '+newtitle)
                        if(str(customtitlevalue)!=newtitle):
                            customtitle.clear()
                            customtitle.send_keys(newtitle)
                            savebutton.click()
                            print("F|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '
                                  +workingtitle)
                            fix_rank()
                            del nametext[:] 
                            time.sleep(10)
                        else:
                            savebutton.click()
                            print("G|"+str(i)+"|"+nametext[0]+"|current custom title: |"+customtitlevalue+'||No change made')
                            fix_rank()
                            del nametext[:] 
                            time.sleep(10)
                    except:
                        cancelbutton.click()
                        print("H|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+
                              '|old working title: '+workingtitle+'|No change made')
                        fix_rank()
                        del nametext[:] 
                        time.sleep(10)
                elif(not box.is_selected()):
                    cancelbutton.click()
                    print("I|"+str(i)+"|"+nametext[0]+'|unmatched working title: |'+workingtitle+'||No change made')
                    fix_rank()
                    del nametext[:] 
                    time.sleep(10)
                    
            elif(len(matchfixedcustom)>0 and box.is_selected()):
                cancelbutton.click()
                print("J|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+'||No change made')
                fix_rank()
                del nametext[:] 
                time.sleep(10)
            else:
                print("K|"+str(i)+"|"+nametext[0]+'|unmatched working title: |'+workingtitle+'||No change made')
                fix_rank()
                del nametext[:] 
                savebutton.click()
                time.sleep(10)
        except NoSuchElementException:
            print(str(i+2)+". ERROR: Mismatched name ("+nametext+") --OR-- ASURITE ("+rowSeries[1]+") is not valid. Please doublecheck.")
            search.clear()
print("Done!")

In [17]:
import pandas as pd
unitlist = pd.read_excel("sirc-to-remove.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
trashbutton = driver.find_element_by_id('edit-asu-dept-mgr-employee-table-remove')
import time
from selenium.webdriver.support.ui import Select
trashlist = []
for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a").text
    try: 
        hrname = unitlist[unitlist['Name']==name]['Name'].values
    except:
        print("error on "+str(i))

In [ ]:
# CHECK ISEARCH AGAINST SPREADSHEET FROM PEOPLESOFT
# This cell checks excel spreadsheet data for the name in the isearch table and creates a list of those
# who are listed in the spreadsheet and are in iSearch, with the option to delete them.

import pandas as pd
unitlist = pd.read_excel("sirc-to-remove.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
trashbutton = driver.find_element_by_id('edit-asu-dept-mgr-employee-table-remove')
import time
from selenium.webdriver.support.ui import Select
trashlist = []
for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a").text
    box = row.find_elements_by_tag_name("td")[0].find_element_by_tag_name("input")
    hrname = unitlist[unitlist['Name']==name]['Name'].values
    if(len(hrname)>0):
        box.click()
        trashlist.append(name)    
print("Delete list:",*trashlist, sep='\n')
# time.sleep(1)
# if(len(trashlist)>0):
#     trashbutton.click()
#     time.sleep(4)
#     confirmdelete = driver.find_element_by_id('edit-submit')
#     confirmdelete.click()
print("Done!")

In [ ]:
# POPULATE FACULTY RANK FIELD
# This cell loops through and checks ALL people CURRENTLY in the selected iSearch unit to see if they are classified as "Faculty."
# Then, it references a spreadsheet to match faculty's current titles and populates the "Faculty Rank" field

import time
import pandas as pd
from selenium.webdriver.support.ui import Select
table = pd.read_excel("faculty-rank-conversion.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
# print(len(rows))
for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    if('Faculty' in row.find_elements_by_tag_name("td")[2].text) or ('Academic' in row.find_elements_by_tag_name("td")[2].text):
        name.click()
        time.sleep(12)
        title = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute("value")
        if title.find('amp;')>0:
            title = title.replace('amp;','')
        rank = table[table['Title']==title]['Faculty Rank'].values
        if len(rank)>0:
            select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
            select.select_by_visible_text(rank[0])
            print("Row"+str(i)+" Success:",name.text,"(Rank:",rank[0]+")")
        else:
            select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
            print("Row"+str(i)+" Error:",name.text+" (Missing Rank:",title+")")
        driver.find_element_by_xpath('//*[@id="edit-save"]').click()
    time.sleep(12)
print("Done!")

In [ ]:
# IMPORT PEOPLE AND POPULATE FACULTY RANK FIELD
# Imports spreadsheet of specific unit, checks iSearch for people that have not been entered, 
# and enters them programatically. Spreadsheet must have two columns: Name, ASURITE
# It also references a spreadsheet to match faculty's current titles and 
# populates the "Faculty Rank" field as they are added

import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
table = pd.read_excel("faculty-rank-conversion.xlsx")
unitlist = pd.read_excel("college-staff-some-subunits.xlsx")
# abbrevlist = pd.read_excel("abbreviations.xlsx")
titlelist = pd.read_excel("titles.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
isearchnames = []
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
search = driver.find_element_by_id('edit-asu-dept-mgr-userpicker')
addbutton = driver.find_element_by_id('edit-asu-dept-mgr-add-to-dept')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    isearchnames.append(name.text)
# print("Names in iSearch:",*isearchnames, sep='\n')

startingrow = 2
for i in range(startingrow-2, unitlist.shape[0]):
    rowSeries = unitlist.iloc[i]
#     print(rowSeries.values)
    try:
        if(isearchnames.index(rowSeries[0])):
            print(str(i+2)+". Exists: "+rowSeries[0])
    except ValueError:
        asurite = rowSeries[1]
        search.clear()
        search.send_keys(asurite)
        addbutton.click()
        time.sleep(14)
        try:
            ftype = Select(driver.find_element_by_xpath('//*[@id="edit-field-employee-type-und"]')).first_selected_option.text
    #         print(ftype)
            fname = driver.find_element_by_xpath('//*[@id="relation-ui-edit-form"]/div/h2')
            customizebox = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
            if customizebox.get_attribute('checked'):
                customizebox.click()
            if (ftype == "Faculty") or (ftype == "Faculty w/Admin Appointment") or (ftype == "Academic Professional") or (ftype == "Academic Prof w/Admin Appointment"):
                title = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute("value")
                if title.find('amp;')>0:
                    title = title.replace('amp;','')
                rank = table[table['Title']==title]['Faculty Rank'].values
                if len(rank)>0:
                    select = Select(driver.find_element_by_xpath('//*[@id="edit-field-faculty-title-und"]'))
                    select.select_by_visible_text(rank[0])
                    print("Success:",fname.text,"(Rank:",rank[0]+")")
                else:
                    print(fname.text+" (Missing Rank:",title+")")
            savebutton = driver.find_element_by_id('edit-save')
            savebutton.click()
            time.sleep(12)
            print("Added: "+rowSeries[0])
        except NoSuchElementException:
            print(str(i+2)+". ERROR: Mismatched name ("+rowSeries[0]+") --OR-- ASURITE ("+rowSeries[1]+") is not valid. Please doublecheck.")
            search.clear()
print("Done!")

In [ ]:
# Find/Replace Title abbreviations
import time
import pandas as pd

titlelist = pd.read_excel("titles.xlsx")
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    nametext = []
    nametext.append(name.text)
#     print(name.text)
#     if('Faculty' in row.find_elements_by_tag_name("td")[2].text) or 
#     ('Academic' in row.find_elements_by_tag_name("td")[2].text):
    name.click()
    time.sleep(10)
    box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
    workingtitle = driver.find_element_by_xpath('//*[@id="edit-working-title"]').get_attribute('value')
#     print('working: '+workingtitle)
    customtitle = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]')
    customtitlevalue= customtitle.get_attribute('value')
#     print('custom: '+customtitlevalue)
    matchworking = titlelist[titlelist['Title']==workingtitle]
#     print('matchworking: '+matchworking)
    matchcustom = titlelist[titlelist['Title']==customtitlevalue]
    matchfixedcustom = titlelist[titlelist['Fixed Title']==customtitlevalue]
#     print('matchcustom: '+matchcustom)
    savebutton = driver.find_element_by_id('edit-save')
    cancelbutton = driver.find_element_by_id('edit-cancel')
    if(str(workingtitle) == "" and not box.is_selected()):
        savebutton.click()
        print("A|"+str(i)+"|"+nametext[0]+'|current working title: |'+workingtitle+'||No change made')
        del nametext[:] 
        time.sleep(10)
    elif(len(matchworking)>0):
        if(not box.is_selected()):
            box.click()
            time.sleep(2)
            savebutton = driver.find_element_by_id('edit-save')
            newtitle = matchworking['Fixed Title'].values[0]
            customtitle.clear()
            customtitle.send_keys(newtitle)
            savebutton.click()
            print("B|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '+workingtitle)
            del nametext[:] 
            time.sleep(10)
        elif(box.is_selected()):
            try:
                newtitle = matchcustom['Fixed Title'].values[0]
                if(customtitle!=newtitle):
                    customtitle.clear()
                    customtitle.send_keys(newtitle)
                    savebutton.click()
                    print("C|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '
                          +workingtitle)
                    del nametext[:] 
                    time.sleep(10)
                else:
                    savebutton.click()
                    print("D|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+'||No change made')
                    del nametext[:] 
                    time.sleep(10)
            except:
                cancelbutton.click()
                print("E|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+
                      '|old working title: '+workingtitle+'|No change made')
                del nametext[:] 
                time.sleep(10)
    elif(len(matchcustom)>0):
        if(box.is_selected()):
            try:
                newtitle = matchcustom['Fixed Title'].values[0]
    #             print('newtitle: '+newtitle)
                if(str(customtitlevalue)!=newtitle):
                    customtitle.clear()
                    customtitle.send_keys(newtitle)
                    savebutton.click()
                    print("F|"+str(i)+"|"+nametext[0]+'|new custom title: |'+newtitle+'|old working title: '
                          +workingtitle)
                    del nametext[:] 
                    time.sleep(10)
                else:
                    savebutton.click()
                    print("G|"+str(i)+"|"+nametext[0]+"|current custom title: |"+customtitlevalue+'||No change made')
                    del nametext[:] 
                    time.sleep(10)
            except:
                cancelbutton.click()
                print("H|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+
                      '|old working title: '+workingtitle+'|No change made')
                del nametext[:] 
                time.sleep(10)
        elif(not box.is_selected()):
            cancelbutton.click()
            print("I|"+str(i)+"|"+nametext[0]+'|unmatched working title: |'+workingtitle+'||No change made')
            del nametext[:] 
            time.sleep(10)
            
    elif(len(matchfixedcustom)>0 and box.is_selected()):
        cancelbutton.click()
        print("J|"+str(i)+"|"+nametext[0]+'|current custom title: |'+customtitlevalue+'||No change made')
        del nametext[:] 
        time.sleep(10)
    else:
        print("K|"+str(i)+"|"+nametext[0]+'|unmatched working title: |'+workingtitle+'||No change made')
        del nametext[:] 
        savebutton.click()
        time.sleep(10)
print("Done!")

In [ ]:
# ORIGINAL Import spreadsheet of specific unit, check iSearch for people that have not been entered, 
# and enter them programatically. Spreadsheet must have two columns: Name, ASURITE
import time
import pandas as pd

unitlist = pd.read_excel("tenure-track.xlsx")
isearchnames = []
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/table/tbody/tr')
search = driver.find_element_by_id('edit-asu-dept-mgr-userpicker')
addbutton = driver.find_element_by_id('edit-asu-dept-mgr-add-to-dept')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a").text
    isearchnames.append(name)
# print("Names in iSearch:",*isearchnames, sep='\n')

excelrow = next(unitlist.iterrows())[1]
for excelrow in unitlist.itertuples():
    try:
        if(isearchnames.index(excelrow[1])):
#             print(str(i)+". Exists: "+excelrow[1])
            pass
    except ValueError:
        asurite = excelrow[2]
        search.send_keys(asurite)
        addbutton.click()
        time.sleep(12)
        savebutton = driver.find_element_by_id('edit-save')
        savebutton.click()
        print(str(i)+". Missing: "+excelrow[1])
        time.sleep(12)
print("Done!")

In [8]:
# Get names and titles of everyone in an iSearch unit
import time
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')

for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    name.click()
    time.sleep(6)
    box = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-source-und-titles"]')
    cancelbutton = driver.find_element_by_id('edit-cancel')
    mname = driver.find_element_by_xpath('//*[@id="relation-ui-edit-form"]/div/h2').text
    titleEdit = driver.find_element_by_xpath('//*[@id="edit-field-employee-title-und-0-value"]').get_attribute('value')
    which = "Edited" if box.is_selected() else "Working"
    print(str(i)+". "+mname+"|"+"Working Title: "+abbrevtitle+"|"+"Edited Title: "+titleEdit+"|selected: "+which)
    cancelbutton.click()
    time.sleep(8)
print("Done!")

NameError: name 'abbrevtitle' is not defined

In [ ]:
rows = driver.find_elements_by_xpath('//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr')
isearchnames = []
for i in range(1,len(rows)+1):
    st = '//*[@id="asu-dept-mgr-employee-table"]/div/table/tbody/tr['+str(i)+']'
    row = driver.find_element_by_xpath(st)
    name = row.find_elements_by_tag_name("td")[1].find_element_by_tag_name("a")
    isearchnames.append(name.text)
print("Names in iSearch:",*isearchnames, sep='\n')